In [13]:
import pandas as pd
import math
import nltk
import gensim
import pyLDAvis.gensim
from gensim import corpora, models
from gensim.models import Word2Vec 

In [2]:
data = pd.read_csv('SentiLex-lem-PT02.txt',header=None)
df = data[0].str.split(';',n=1,expand=True)
df = df[0].str.split('.',n=1,expand=True)
data['expressao'] = df[0]
data['PoS'] = df[1]

df = data[0].str.split(':',n=1,expand=True)
df = df[0].str.split(';',n=1,expand=True)

data['TG'] = df[1]

df = data[0].str.split(';',n=2,expand=True)
df = df[2].str.split(';ANOT',n=2,expand=True)
data['POL'] = df[0]

df = data[0].str.split(';',n=3,expand=True)
data['ANOT'] = df[3]

df = data['POL'].str.split(';',n=1,expand=True)
data['POL'] = df[0]
data['POL2'] = df[1]


In [3]:
# formata dataframe
data['PoS'] = data['PoS'].str.replace('PoS=','')
data['TG'] = data['TG'].str.replace('TG=','')
data['POL'] = data.POL.replace(regex=['.*='],value='')
data['POL2'] = data.POL2.replace(regex=['.*='],value='')
data['ANOT'] = data.ANOT.replace(regex=['P.*;'],value='')
data['ANOT'] = data['ANOT'].str.replace('ANOT=','')
data = data.drop(columns=[0])

df = data['ANOT'].str.split(';',n=1,expand=True)
data['ANOT'] = df[0]
data['REV'] = df[1]
data['REV'] = data['REV'].str.replace('REV=','')
senti_data = data
senti_data.head(30)


,expressao,PoS,TG,POL,ANOT,POL2,REV
0,à-vontade,N,HUM,1,MAN,None,None
1,abafado,Adj,HUM,-1,JALC,None,None
2,abafante,Adj,HUM,-1,MAN,None,None
3,abaixado,Adj,HUM,-1,JALC,None,None
4,abalado,Adj,HUM,-1,JALC,None,None
5,abalizado,Adj,HUM,1,JALC,None,None
6,abalroado,Adj,HUM,-1,MAN,None,None
7,abalroar,V,HUM,1,MAN,-1,None
8,abanar,V,HUM,1,MAN,-1,None
9,abandalhado,Adj,HUM,-1,MAN,None,None


In [4]:
tweet_data = pd.read_csv('final_set.csv',usecols=['text'])
tweet_data

,text
0,#b17 neles
1,deus proteja capitao #bolsonaro17
2,#elesim god help us
3,voto @jairbolsonaro #elesim #mbl #ordemeprog...
4,parece eleicao domingo vai ser igual crianca b...
...,...
138,#haddadnaoecristao latim : clube bilderberg
139,fara passeio curitiba jean willis ta camara f...
140,olsominions usando tag tipo defendem cara apoi...
141,#haddadnaoecristao hora fazer foto fake respe...


In [5]:
def get_polarity(tokens):
    df = senti_data[senti_data['expressao'].isin(tokens)]
    return df

In [6]:
df_pol = pd.DataFrame() #creates a new dataframe that's empty
for index, row in tweet_data.iterrows():
    df = get_polarity(nltk.word_tokenize(row['text']))
    if df.empty == False:
        df_pol = df_pol.append(df, ignore_index=True)
    

In [7]:
df_pol['POL'] = df_pol['POL'].astype(str).astype(int)

df_pol

,expressao,PoS,TG,POL,ANOT,POL2,REV
0,igual,Adj,HUM,0,JALC,None,None
1,democracia,N,HUM,1,MAN,None,None
2,liberal,Adj,HUM,0,MAN,None,None
3,ganhar,V,HUM,1,MAN,-1,None
4,perder,V,HUM,-1,MAN,0,None
5,derrubar,V,HUM,1,MAN,-1,None
6,intimidar,V,HUM,1,MAN,-1,None
7,consciente,Adj,HUM,1,MAN,None,None
8,melhor,Adj,HUM,0,JALC,None,None
9,responsabilidade,N,HUM,1,MAN,None,None


In [8]:
# positivas
print(df_pol.loc[df_pol['POL'] == 1])
pos = df_pol.loc[df_pol['POL'] == 1].sum()['POL']
pos

           expressao  PoS   TG  POL  ANOT  POL2   REV
1         democracia    N  HUM    1   MAN  None  None
3             ganhar    V  HUM    1   MAN    -1  None
5           derrubar    V  HUM    1   MAN    -1  None
6          intimidar    V  HUM    1   MAN    -1  None
7         consciente  Adj  HUM    1   MAN  None  None
9   responsabilidade    N  HUM    1   MAN  None  None
13         maravilha    N  HUM    1   MAN  None  None
14           verdade    N  HUM    1   MAN  None  None
16         gentileza    N  HUM    1   MAN  None  None
23           verdade    N  HUM    1   MAN  None  None
24        conhecedor  Adj  HUM    1  JALC  None  None
26           verdade    N  HUM    1   MAN  None  None
27      bem-sucedido  Adj  HUM    1   MAN  None  None
29           direito  Adj  HUM    1  JALC  None   Amb
31              vivo  Adj  HUM    1  JALC  None  None
34           avisado  Adj  HUM    1  JALC  None   Amb
38           honrado  Adj  HUM    1   MAN  None  None
48            vencer    V  H

18

In [9]:
# negativas
print(df_pol.loc[df_pol['POL'] == -1])
neg = df_pol.loc[df_pol['POL'] == -1].sum()['POL']
neg

     expressao  PoS   TG  POL  ANOT  POL2   REV
4       perder    V  HUM   -1   MAN     0  None
10      babaca  Adj  HUM   -1  JALC  None  None
11       preso  Adj  HUM   -1  JALC  None  None
12     partido  Adj  HUM   -1   MAN  None   Amb
17  hipocrisia    N  HUM   -1   MAN  None  None
19     mentira    N  HUM   -1   MAN  None  None
20    corrupto  Adj  HUM   -1   MAN  None  None
21   mentiroso  Adj  HUM   -1   MAN  None  None
22     partido  Adj  HUM   -1   MAN  None   Amb
25     enganar    V  HUM   -1   MAN     0  None
30       farsa    N  HUM   -1   MAN  None  None
32     exibido  Adj  HUM   -1  JALC  None   Amb
36    corrupto  Adj  HUM   -1   MAN  None  None
37       farsa    N  HUM   -1   MAN  None  None
41  desgoverno    N  HUM   -1   MAN  None  None
42      perder    V  HUM   -1   MAN     0  None
43       pobre  Adj  HUM   -1   MAN  None  None
44      perder    V  HUM   -1   MAN     0  None
45     acabado  Adj  HUM   -1   MAN  None   Amb
46      amoral  Adj  HUM   -1   MAN  Non

-24

In [10]:
#select column to convert to list here
tweets = tweet_data["text"].tolist()

tweets_text = []
for index, row in tweet_data.iterrows():
    l = row['text'].split(' ')
    while("" in l) : 
        l.remove("") 
    tweets_text.append(l)
    
# tweets_text

In [11]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(tweets_text)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in tweets_text]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word = dictionary, passes=20,random_state=1)
print(ldamodel.print_topics(num_topics=4, num_words=4))

[(0, '0.018*"#b17" + 0.017*"#elesim" + 0.017*"vai" + 0.015*"#bolsonaro17"'), (1, '0.020*"#haddadnaoecristao" + 0.008*"#bolsonaro17" + 0.008*"pra" + 0.008*":"'), (2, '0.037*"#b17" + 0.016*"#bolsonaropresidente17" + 0.012*"nao" + 0.012*"#haddadnaoecristao"'), (3, '0.032*"#bolsonaro17" + 0.017*"#bolsonaropresidente" + 0.014*"@jairbolsonaro" + 0.014*"acima"')]


In [12]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/home/brenda/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [15]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(tweets_text, min_count = 1, size = 100, window = 5) 

In [22]:
# Print results 
print("Cosine similarity - CBOW : ", +
    model1.similarity('haddad', 'tortura')) 

Cosine similarity - CBOW :  -0.12778848


/home/brenda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
